In [2]:
import pytorch_lightning as pl
from phantoms.models.retrieval.gnn_retrieval_model import GNNRetrievalModel  # Existing small model
from phantoms.models.retrieval.gnn_retrieval_model_skip_connection import GNNRetrievalSkipConnections  # New large model
from massspecgym.data.data_module import MassSpecDataModule
from massspecgym.data.transforms import MolFingerprinter
from massspecgym.featurize import SpectrumFeaturizer
from pytorch_lightning.loggers import TensorBoardLogger, WandbLogger
import wandb
from massspecgym.data.datasets import MSnRetrievalDataset, MSnDataset
import yaml
import argparse
import shutil
import os
import numpy as np
import random
import torch
from datetime import datetime
from phantoms.utils.data import save_embeddings
from phantoms.optimizations.training import set_global_seeds
from phantoms.utils.parser import validate_config, train_model, extract_and_save_embeddings

In [3]:
pl.seed_everything(42)

config_path = "/Users/macbook/CODE/PhantoMS/phantoms/models/retrieval/configs/config_skip_connection.yml"
# Load configuration
with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

Seed set to 42


# Run one experiment

In [4]:
set_global_seeds(42)

config_path = "/Users/macbook/CODE/PhantoMS/phantoms/models/retrieval/configs/config_skip_connection.yml"
# Load configuration
with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

validate_config(config)

experiment_folder = '/Users/macbook/CODE/PhantoMS/experiments_run'

timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

experiment_folder = os.path.join(experiment_folder, f'{timestamp}_{config["experiment_base_name"]}')

In [5]:
# Define different cut_tree_at_level values
cut_tree_level = None


# Train the model (this will also initialize the data module, loggers, callbacks, etc.)
train_model(config, experiment_folder, config_path, cut_tree_level)


Starting training for /Users/macbook/CODE/PhantoMS/experiments_run/2025-02-04_14-10-26_name
Using retrieval task/model.
Total valid indices: 105
MSnRetrievalDataset length: 105


/Users/macbook/UTILS/anaconda3/envs/phantoms_env/lib/python3.11/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Users/macbook/UTILS/anaconda3/envs/phantoms_env/lib/python3.11/site-packages/pytorch_lightning/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
wandb: Currently logged in as: jozefov (jozefov-iocb-prague). Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.



  | Name       | Type                        | Params | Mode 
-------------------------------------------------------------------
0 | gcn_layers | ModuleList                  | 16.6 M | train
1 | head       | SkipConnectionRetrievalHead | 37.8 M | train
2 | loss_fn    | MSELoss                     | 0      | train
-------------------------------------------------------------------
54.4 M    Trainable params
0         Non-trainable params
54.4 M    Total params
217.596   Total estimated model params size (MB)
54        Modules in train mode
0         Modules in eval mode


Train dataset size: 82
Val dataset size: 6


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

...

/Users/macbook/UTILS/anaconda3/envs/phantoms_env/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     test_hit_rate@1                0.5
    test_hit_rate@20               0.75
     test_hit_rate@5                0.5
        test_loss           0.02292540669441223
       test_mces@1                 21.0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Model saved to /Users/macbook/CODE/PhantoMS/experiments_run/2025-02-04_14-10-26_name/checkpoints/final_model.ckpt


epoch,▁▅█
test_hit_rate@1,▁
test_hit_rate@20,▁
test_hit_rate@5,▁
test_loss,▁
test_mces@1,▁
trainer/global_step,▁▆█
val_hit_rate@1,▁▁
val_hit_rate@20,█▁
val_hit_rate@5,█▁
val_loss,▁█


Configuration saved to /Users/macbook/CODE/PhantoMS/experiments_run/2025-02-04_14-10-26_name/configs/config_skip_connection.yml


In [6]:
extract_and_save_embeddings(config, cut_tree_level, experiment_folder)


Extracting embeddings for /Users/macbook/CODE/PhantoMS/experiments_run/2025-02-04_14-10-26_name


/Users/macbook/UTILS/anaconda3/envs/phantoms_env/lib/python3.11/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


Total valid indices: 105
MSnRetrievalDataset length: 105
Test dataset size: 17
Embeddings saved to /Users/macbook/CODE/PhantoMS/experiments_run/2025-02-04_14-10-26_name/embeddings


# Train all

In [8]:
import wandb
import yaml
import os
from datetime import datetime
import time
from typing import List, Optional

from phantoms.utils.parser import validate_config, train_model, extract_and_save_embeddings
from phantoms.optimizations.training import set_global_seeds

def run_all_experiments(config_dir: str,
                        experiment_parent_dir: str,
                        config_files: List[str],
                        cut_tree_levels: Optional[List[int]],
                        wandb_project_name: str):
    """
    Iterate over all configuration files and tree levels to run experiments.

    Args:
        config_dir (str): Directory containing configuration YAML files.
        experiment_parent_dir (str): Parent directory to store all experiments.
        config_files (List[str]): List of configuration YAML filenames.
        cut_tree_levels (Optional[List[int]]): List of cut_tree_at_level values.
        wandb_project_name (str): Name of the wandb project.
    """
    # Set global seeds for reproducibility
    set_global_seeds(42)

    # Create the parent experiment directory if it doesn't exist
    os.makedirs(experiment_parent_dir, exist_ok=True)

    # Iterate over each configuration file
    for config_file in config_files:
        config_path = os.path.join(config_dir, config_file)

        if not os.path.exists(config_path):
            print(f"Configuration file {config_path} does not exist. Skipping.")
            continue

        # Load the configuration
        with open(config_path, 'r') as f:
            config = yaml.safe_load(f)

        # Validate the configuration
        try:
            validate_config(config)
        except ValueError as ve:
            print(f"Configuration validation error in {config_file}: {ve}")
            continue

        # Iterate over each tree level
        for level in cut_tree_levels:
            timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
            # Adding a slight sleep to ensure different timestamps
            time.sleep(1.0)

            # Define a unique experiment folder name
            config_name = os.path.splitext(os.path.basename(config_file))[0]
            experiment_folder_name = f"{config_name}_cut_tree_{level}_{timestamp}"
            experiment_folder = os.path.join(experiment_parent_dir, experiment_folder_name)

            # Print experiment details for debugging
            print(f"\nRunning Experiment: {experiment_folder_name}")
            print(f"W&B Project: {wandb_project_name}")
            print(f"Cut Tree Level: {level}")

            # Create the experiment folder
            os.makedirs(experiment_folder, exist_ok=True)

            # Modify the config dict's 'trainer.checkpoint_dir' to point to experiment_folder subdirectories
            config['trainer']['checkpoint_dir'] = os.path.join(experiment_folder, 'checkpoints')

            # Optionally, modify 'experiment_base_name' to include the experiment_folder name or set to a unified value
            config['experiment_base_name'] = 'experiment_trial'  # Or any desired base name

            # Update W&B project name
            config['wandb']['project'] = wandb_project_name

            # Train the model
            train_model(config, experiment_folder, config_path, level)

            # Extract and save embeddings
            extract_and_save_embeddings(config, level, experiment_folder)

            # Finish the W&B run to ensure it's properly logged
            wandb.finish()

    print("\nAll experiments completed successfully.")

In [9]:
# Define parameters
config_directory = '/Users/macbook/CODE/PhantoMS/phantoms/models/retrieval/configs' 
experiment_parent_directory = '/Users/macbook/CODE/PhantoMS/experiments_run/new_gnn_retrieval_cut'  

configuration_files = [
    'config_skip_connection.yml',
    'config_skip_connection_bonus.yml',
    'config_skip_connection_dreams.yml',
    'config_skip_connection_dreams_bonus.yml'
]
tree_levels = [0, 1, 2, 3]

wandb_project_name = 'new_gnn_retrieval_cut'

In [10]:
run_all_experiments(config_dir=config_directory,
                    experiment_parent_dir=experiment_parent_directory,
                    config_files=configuration_files,
                    cut_tree_levels=tree_levels,
                    wandb_project_name=wandb_project_name)


Running Experiment: config_skip_connection_cut_tree_0_2025-02-04_14-25-28
W&B Project: new_gnn_retrieval_cut
Cut Tree Level: 0

Starting training for /Users/macbook/CODE/PhantoMS/experiments_run/new_gnn_retrieval_cut/config_skip_connection_cut_tree_0_2025-02-04_14-25-28
Using retrieval task/model.
Total valid indices: 105
MSnRetrievalDataset length: 105


/Users/macbook/UTILS/anaconda3/envs/phantoms_env/lib/python3.11/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Users/macbook/UTILS/anaconda3/envs/phantoms_env/lib/python3.11/site-packages/pytorch_lightning/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.



  | Name       | Type                        | Params | Mode 
-------------------------------------------------------------------
0 | gcn_layers | ModuleList                  | 16.6 M | train
1 | head       | SkipConnectionRetrievalHead | 37.8 M | train
2 | loss_fn    | MSELoss                     | 0      | train
-------------------------------------------------------------------
54.4 M    Trainable params
0         Non-trainable params
54.4 M    Total params
217.596   Total estimated model params size (MB)
54        Modules in train mode
0         Modules in eval mode


Train dataset size: 82
Val dataset size: 6


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

...

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     test_hit_rate@1                0.0
    test_hit_rate@20               0.75
     test_hit_rate@5               0.25
        test_loss          0.049181342124938965
       test_mces@1                 13.75
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Model saved to /Users/macbook/CODE/PhantoMS/experiments_run/new_gnn_retrieval_cut/config_skip_connection_dreams_bonus_cut_tree_3_2025-02-04_14-34-56/checkpoints/final_model.ckpt


epoch,▁▅█
test_hit_rate@1,▁
test_hit_rate@20,▁
test_hit_rate@5,▁
test_loss,▁
test_mces@1,▁
trainer/global_step,▁▆█
val_hit_rate@1,▁▁
val_hit_rate@20,█▁
val_hit_rate@5,█▁
val_loss,▁█


Configuration saved to /Users/macbook/CODE/PhantoMS/experiments_run/new_gnn_retrieval_cut/config_skip_connection_dreams_bonus_cut_tree_3_2025-02-04_14-34-56/configs/config_skip_connection_dreams_bonus.yml

Extracting embeddings for /Users/macbook/CODE/PhantoMS/experiments_run/new_gnn_retrieval_cut/config_skip_connection_dreams_bonus_cut_tree_3_2025-02-04_14-34-56


/Users/macbook/UTILS/anaconda3/envs/phantoms_env/lib/python3.11/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


Total valid indices: 105
MSnRetrievalDataset length: 105
Test dataset size: 17
Embeddings saved to /Users/macbook/CODE/PhantoMS/experiments_run/new_gnn_retrieval_cut/config_skip_connection_dreams_bonus_cut_tree_3_2025-02-04_14-34-56/embeddings

All experiments completed successfully.
